## Неделя 12. Домашнее задание по отбору признаков.

### Теоретические вопросы:
1) Какая группа методов отбора признаков является наиболее вычислительно затратной?
    1. Filter methods (методы фильтрации)
    2. Embedded methods (встроенные методы)
    3. Wrapper methods (методы обёртки)
    
2) Какая группа методов отбора признаков используется ДО обучения модели?
    1. Filter methods (методы фильтрации)
    2. Embedded methods (встроенные методы)
    3. Wrapper methods (методы обёртки)
    
3) Какой из нижеперечисленных методов отбора признаков относится к методам фильтрации?
    1. Recursive feature elimination
    2. LASSO
    3. Метод на основе chi-squared
    
4) Какой из нижеперечисленных методов отбора признаков относится к встроенным методам?
    1. Recursive feature elimination
    2. LASSO
    3. Метод на основе chi-squared
    
5) Какой из нижеперечисленных методов отбора признаков относится к методам обёртки?
    1. Recursive feature elimination
    2. LASSO
    3. Метод на основе chi-squared

**Answer:**
1. 3
2. 1
3. 3
4. 2
5. 1

### Практическая часть.

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2, RFE

import seaborn as sns

**Описание контекста задачи**
В некоторой компании занимающейся услугами мобильной связи для клиентов доступны дополнительные платные услуги A, B и C,которые взаимоисключают друг друга. Руководство компании хочет построить модель, которая будет для каждого клиента определять наиболее подходящую ему услугу для таргетированной рекламы этих услуг внутри компании.
**Описание датасета**
Каждая строка в датасете определяет одного клиента компани и дополнительную услугу.
В датасете имеются следующие колонки:
- age: возраст клиента
- No: номер договора
- month_from_start: количество месяцев с даты заключения договора
- base_cost: стоимость основного тарифа в рублях
- block_count: количество блокировок
- is_used_app: использует ли мобильное приложение
- month_incall_mean: среднее количество входящих звонков в месяц
- month_outcall_mean: среднее количество исходящих звонков в месяц
- add_service: дополнительная платная услуга

In [2]:
df = pd.read_csv('mobile_service.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   age                 1300 non-null   float64
 1   No                  1300 non-null   int64  
 2   month_from_start    1300 non-null   int64  
 3   base_cost           1300 non-null   int64  
 4   block_count         1300 non-null   int64  
 5   is_used_app         1300 non-null   int64  
 6   month_incall_mean   1300 non-null   float64
 7   month_outcall_mean  1300 non-null   float64
 8   add_service         1300 non-null   int64  
dtypes: float64(3), int64(6)
memory usage: 91.5 KB


In [4]:
X = df.drop('add_service', axis=1).copy()
y = df['add_service'].copy()

**Вопрос 6.** Какие два признака будут исключены из датасета если использовать метод отбора признаков, основанный на статистике хи-квадрат c конечным количеством признаков 6 (9 всего - 1 таргет - 2 признака)? В ответе выберите нужные признаки.

In [7]:
# код тут
sel_k_best = SelectKBest(chi2, k=6)
sel_k_best.fit(X, y)
X_selected = pd.DataFrame(sel_k_best.transform(X), 
                         columns = [a for a,b in zip(df.columns, sel_k_best.get_support()) if b])
chi2_cols = X_selected.columns
set(X.columns)-set(chi2_cols)

{'base_cost', 'block_count'}

**Вопрос 7.** Какие два признака будут присутствовать в датасете если использовать метод отбора признаков RFE, с параметрами estimator=sklearn.tree.DecisionTreeClassifier и step=1 c конечным количеством признаков 2 с различными random_state? В ответе выберите нужные признаки.

In [12]:
# код тут
estimator = DecisionTreeClassifier()
rfe = RFE(estimator, n_features_to_select=2, step=1)
rfe.fit(X, y)
X_selected = pd.DataFrame(rfe.transform(X), 
                         columns = [a for a,b in zip(df.columns, rfe.get_support()) if b])
rfe_cols = X_selected.columns
rfe_cols

Index(['age', 'month_outcall_mean'], dtype='object')

**Вопрос 8.** Какие два признака в датасете имеют коэффициент корреляции > 0.5 (между собой, не с таргетом)? В ответе выберите нужные признаки.

In [8]:
# код тут
X.corr()

,age,No,month_from_start,base_cost,block_count,is_used_app,month_incall_mean,month_outcall_mean
age,1.000000,0.020843,0.016129,-0.005241,0.011103,0.046018,-0.044111,-0.053912
No,0.020843,1.000000,-0.075299,0.040723,-0.026112,-0.022328,0.007470,0.005794
month_from_start,0.016129,-0.075299,1.000000,0.007472,0.001757,-0.013250,0.031683,-0.029484
base_cost,-0.005241,0.040723,0.007472,1.000000,0.012973,0.007132,0.028577,0.034327
block_count,0.011103,-0.026112,0.001757,0.012973,1.000000,0.012788,0.030286,0.012282
is_used_app,0.046018,-0.022328,-0.013250,0.007132,0.012788,1.000000,-0.017679,0.010767
month_incall_mean,-0.044111,0.007470,0.031683,0.028577,0.030286,-0.017679,1.000000,0.567645
month_outcall_mean,-0.053912,0.005794,-0.029484,0.034327,0.012282,0.010767,0.567645,1.000000


**Answer:** month_outcall_mean and month_incall_mean